This version is better because it also outputs the diversity and productivity (for each document)

Function to calculate <u>diversity</u> (Shannon's entropy) of a sentence

In [1]:
import scipy.stats as stats

def sentence_diversity_calc(tags):
  """Calculates the Shannon entropy of a list of POS tag pairs.

  Args:
    pairs: A list of POS tag pairs (tuples).

  Returns:
    The Shannon entropy of the pairs.
  """

  pairs = []
  for i in range(len(tags) - 1):
    pairs.append((tags[i], tags[i+1]))
  
  # Count occurrences of each pair
  pair_counts = {}
  for pair in pairs:
    pair_counts[pair] = pair_counts.get(pair, 0) + 1

  # Calculate probabilities
  total_pairs = sum(pair_counts.values())
  probabilities = [count / total_pairs for count in pair_counts.values()]

  # Calculate entropy using SciPy's entropy function
  H = stats.entropy(probabilities, base=2)  # Base 2 for bits

  return H

Function to caclulate the <u>productivity</u> of each sentence

In [2]:
import scipy.stats as stats

def sentence_productivity_calc(words, tags):
  """Calculates the productivity of a sentence based on word and tag data.

  Args:
    words: A list of words in the sentence.
    tags: A list of corresponding POS tags.

  Returns:
    The productivity of the sentence.
  """

  # Calculate joint entropy H(W, T)
  word_tag_pairs = list(zip(words, tags))
  pair_counts = {}
  for pair in word_tag_pairs:
    pair_counts[pair] = pair_counts.get(pair, 0) + 1
  total_pairs = sum(pair_counts.values())
  probabilities = [count / total_pairs for count in pair_counts.values()]
  H_WT = stats.entropy(probabilities, base=2)

  # Calculate entropy of tags H(T)
  tag_counts = {}
  for tag in tags:
    tag_counts[tag] = tag_counts.get(tag, 0) + 1
  total_tags = sum(tag_counts.values())
  tag_probabilities = [count / total_tags for count in tag_counts.values()]
  H_T = stats.entropy(tag_probabilities, base=2)

  # Calculate conditional entropy H(W | T)
  H_WT_given_T = H_WT - H_T

  # Calculate productivity
  productivity = H_WT_given_T + 1

  return productivity

Function to calculate the document complexity

In [3]:
def document_complexity_calc(sentences, doc):
    """Calculates the document complexity, diversity, and productivity.

    Args:
        sentences: A list of sentences (each sentence as a list of words).
        doc: The entire document object.

    Returns:
        A tuple containing:
        - Document complexity
        - Average document diversity
        - Average document productivity
    """

    N = len(sentences)
    total_complexity = 0
    total_diversity = 0
    total_productivity = 0

    for sentence in sentences:
        sen_words = []
        sen_pos = []

        for word in sentence.words:
            if word.upos != "PUNCT":
                sen_words.append(word.text.lower())
                sen_pos.append(word.xpos)

        # Calculate D(si) and P(si) for the current sentence
        diversity = sentence_diversity_calc(sen_pos)
        productivity = sentence_productivity_calc(sen_words, sen_pos)

        # Update totals
        total_complexity += diversity * productivity
        total_diversity += diversity
        total_productivity += productivity

    # Calculate averages
    document_complexity = total_complexity / N
    average_diversity = total_diversity / N
    average_productivity = total_productivity / N

    return document_complexity, average_diversity, average_productivity

In [4]:
import stanza
import ipywidgets as widgets
import pandas as pd
from IPython.display import display

# Initialize the Stanza pipeline
nlp = stanza.Pipeline("en")

# Create a title widget
title_widget = widgets.HTML(value="<h1>Construction Complexity Calculator</h1>")

# Create a description widget
description_widget = widgets.HTML(value="<p>This tool calculates the complexity measure described in Nelson (2024).</p>")

# Create a combined instructions widget
instructions_widget = widgets.HTML(value="<p>For multiple texts, click \"upload\" and select the files. For one text, paste it into the box. When your texts are ready, click \"Process Input\".</p>")

# Create a file upload widget
file_upload_widget = widgets.FileUpload(
    accept=".txt",  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)

# Create a text box widget
text_box_widget = widgets.Textarea(
    placeholder='Copy and paste text here',
    description='Input text:',
    layout=widgets.Layout(height='300px', width='600px')
)

# Create an output widget to display the result
output_widget = widgets.Output()

# Create a progress bar widget
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Progress:',
    bar_style='',  # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

# Define the function to process the input files and create a CSV file
def process_input(files, text):
    if files:
        results = []
        total_files = len(files)
        progress_bar.max = total_files
        
        for i, uploaded_file in enumerate(files):
            content = bytes(uploaded_file['content']).decode('utf-8')
            doc = nlp(content)
            complexity, avg_diversity, avg_productivity = document_complexity_calc(doc.sentences, doc)
            results.append({'filename': uploaded_file['name'],
                            'complexity': complexity,
                            'diversity': avg_diversity,
                            'productivity': avg_productivity})
            
            # Update progress bar
            progress_bar.value = i + 1
        
        df = pd.DataFrame(results)
        csv_filename = 'document_complexity_scores.csv'
        df.to_csv(csv_filename, index=False)
        
        output_widget.clear_output()
        output_widget.append_stdout(f"Complexity scores saved to {csv_filename}")
        progress_bar.value = 0  # Reset progress bar
    
    elif text:
        progress_bar.max = 1
        progress_bar.value = 0
        
        doc = nlp(text)
        complexity, avg_diversity, avg_productivity = document_complexity_calc(doc.sentences, doc)
        output_widget.clear_output()
        output_widget.append_stdout(f"""
        Complexity score: {complexity}
        Diversity: {avg_diversity}
        Productivity: {avg_productivity}
        """)
        
        progress_bar.value = 1  # Complete progress bar
        progress_bar.value = 0  # Reset progress bar

# Attach a button to process the input
process_button = widgets.Button(description="Process Input")
process_button.on_click(lambda b: process_input(file_upload_widget.value, text_box_widget.value))

# Create a reference widget with a hyperlink
reference_widget = widgets.HTML(value="""
<p>Read the Open Access paper for full details about the complexity measure:</p>
<p>Nelson, R. (2024). Using constructions to measure developmental language complexity. Cognitive Linguistics. <a href="https://doi.org/10.1515/cog-2023-0062">https://doi.org/10.1515/cog-2023-0062</a></p>
""")

# Display the widgets (no centering for button)
display(title_widget, description_widget, instructions_widget, file_upload_widget, text_box_widget, process_button, output_widget, reference_widget, progress_bar)


2024-10-16 07:21:44 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-10-16 07:21:45 INFO: Downloaded file to C:\Users\admin\stanza_resources\resources.json
2024-10-16 07:21:45 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2024-10-16 07:21:45 INFO: Using device: cpu
2024-10-16 07:21:45 INFO: Loading: tokenize
c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\stanza\models\tokenization\trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malic

HTML(value='<h1>Construction Complexity Calculator</h1>')

HTML(value='<p>This tool calculates the complexity measure described in Nelson (2024).</p>')

HTML(value='<p>For multiple texts, click "upload" and select the files. For one text, paste it into the box. W…

...

Textarea(value='', description='Input text:', layout=Layout(height='300px', width='600px'), placeholder='Copy …

Button(description='Process Input', style=ButtonStyle())

Output()

HTML(value='\n<p>Read the Open Access paper for full details about the complexity measure:</p>\n<p>Nelson, R. …

IntProgress(value=0, description='Progress:')